In [1]:
import numpy as np
import pandas as pd
import sklearn
import cython
import talib

from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

from Functions import DataReady, KijunSen

/usr/lib/python3/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv("/home/porsche/Desktop/Training_data/5minData/ATOMUSDT5min.csv")

DataReady(data)

,Open,High,Low,Close,Volume
datetime,,,,,
2024-03-11 02:00:00,13.190,13.233,13.179,13.219,10036.8
2024-03-11 02:05:00,13.219,13.276,13.217,13.243,20072.7
2024-03-11 02:10:00,13.243,13.262,13.201,13.262,8939.3
2024-03-11 02:15:00,13.262,13.264,13.203,13.205,9896.9
2024-03-11 02:20:00,13.205,13.209,13.150,13.167,8269.5
...,...,...,...,...,...
2024-04-02 19:05:00,10.943,11.030,10.931,11.013,22089.6
2024-04-02 19:10:00,11.013,11.031,10.992,11.031,16393.6
2024-04-02 19:15:00,11.031,11.031,10.950,10.976,10379.1


In [ ]:
class MachineLearningStrategyV1(Strategy):
    
    # Add variables to the strategy
    atr_lenght = 14
    times_atr_tp = 3
    times_atr_sl = 2
    upper_ema_lenght = 50
    lower_ema_lenght = 200
    stoch_rsi_lenght = 14
    kijun_sen_period = 26
    kijun_sen_price_range = 0.2
    # 15×5×5×50×50×15×20×10 = 2812500000

    def init(self):
        # Indicator code
        self.atr = self.I(talib.ATR, self.data.High, self.data.Low, self.data.Close, self.atr_lenght)
        self.ema_upper = self.I(talib.EMA, self.data.Close, self.upper_ema_lenght)
        self.ema_lower = self.I(talib.EMA, self.data.Close, self.lower_ema_lenght)
        self.stoch_rsi = self.I(talib.STOCHRSI, self.data.Close, self.stoch_rsi_lenght)
        self.kijun_sen = self.I(KijunSen, data, self.kijun_sen_period)
        self.kijun_sen_to_price = self.data.Close - self.kijun_sen

        
    def next(self):
        if crossover(self.ema_lower, self.ema_upper) and self.kijun_sen_to_price < self.kijun_sen_price_range:
            self.buy(sl = self.data.Close - (self.atr*self.times_atr_sl), tp = self.data.Close + (self.atr*self.times_atr_tp))
        

    
bt = Backtest(data, MachineLearningStrategyV1, cash = 10000)

stats = bt.optimize(
    upper_ema_lenght = range(20, 100, 5),
    lower_ema_lenght = range(100, 200, 5),
    kijun_sen_period = range(10, 40, 1),
    times_atr_tp = range(1, 5, 1),
    times_atr_sl = range(1, 5, 1),
    atr_lenght = range(5, 30, 1),
    kijun_sen_price_range = range(0.00, 1, 0.01),
    maximize = "Sharpe Ratio"
)

print(stats)

bt.plot()